# Reddit 🕷️ Spider for PlayStation Comments 🎮

**https://www.reddit.com/robots.txt**

- https://www.reddit.com/r/playstation/
- https://www.reddit.com/r/PlaystationPortal/
- https://www.reddit.com/r/PlayStationPlus/
- https://www.reddit.com/r/PS4/
- https://www.reddit.com/r/PS5/
- https://www.reddit.com/r/PSVR/

**Note** Decided to use the `/top/?t=year` URL param to help filter the noise. I will then take the top 1,000 posts for the past year. 

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import time

In [17]:
from io import StringIO
from dotenv import load_dotenv
from dotenv import dotenv_values
from dotenv import find_dotenv
import os

In [18]:
load_dotenv(find_dotenv())

True

In [19]:
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")

In [21]:
def login_to_reddit(driver, username, password):
    driver.get("https://www.reddit.com/login/")
    time.sleep(2)  
    username_field = driver.find_element(By.ID, 'loginUsername')
    username_field.send_keys(username)
    password_field = driver.find_element(By.ID, 'loginPassword')
    password_field.send_keys(password)
    login_button = driver.find_element(By.XPATH, "//button[contains(@class, 'AnimatedForm__submitButton')]")
    login_button.click()
    time.sleep(2)  # Note: I tried 1 sec and it's too short maybe my wifi speed lol 

In [25]:
def expand_comments(driver):
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div")))

        while True:
            try:
                more_replies_button = driver.find_element(By.XPATH, "//*[contains(text(), 'more replies')]")
                ActionChains(driver).move_to_element(more_replies_button).click().perform()
                
                time.sleep(2)
            except NoSuchElementException:
                break
            except TimeoutException:
                break

    except Exception as e:
        print(f"Error in expanding comments: {e}")

In [23]:
def extract_comments(driver, article):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    comment_divs = soup.find_all('div', attrs={'data-testid': 'comment'})

    data = []
    for div in comment_divs:
        header_div = div.find_previous_sibling('div', attrs={'data-testid': 'post-comment-header'})

        if header_div:
            author_element = header_div.find('a', attrs={'data-testid': 'comment_author_link'})
            author = author_element.get_text(strip=True) if author_element else 'Null'

            timestamp_element = header_div.find('a', attrs={'data-testid': 'comment_timestamp'})
            timestamp = timestamp_element.get_text(strip=True) if timestamp_element else 'Null'
        else:
            author = 'Null'
            timestamp = 'Null'

        comment_text_element = div.find('div', class_='RichTextJSON-root') 
        comment_text = comment_text_element.get_text(strip=True) if comment_text_element else 'Null'

        data.append({
            'Article': article,
            'Author': author,
            'Timestamp': timestamp,
            'Comment': comment_text,
        })

    return data

In [24]:
def scrape_reddit_comments(file_path, start_row, batch_size):
    df_links = pd.read_csv(file_path)
    all_comments = []
    end_row = start_row + batch_size

    login_to_reddit(driver, username, password)

    for index, row in enumerate(df_links[start_row:end_row].itertuples(), start=start_row):
        link = row.Link
        print(f"Processing {index + 1}/{end_row}: {link}")

        try:
            driver.get(link)
            expand_comments(driver)  
            comments_data = extract_comments(driver, link)
            all_comments.extend(comments_data)
        except Exception as e:
            print(f"Error processing {link}: {e}")
            continue  

    return pd.DataFrame(all_comments)

# The Main Spider 🕷️

In [98]:
driver = webdriver.Chrome()

start_row = 0
batch_size = 2

file_path = '../../Data/Raw Pulls /RAW_rPlaystation_reddit_titles.csv'

df_comments = scrape_reddit_comments(file_path, start_row, batch_size)
print(df_comments)

driver.quit()

Processing 1/2: https://www.reddit.com/r/playstation/comments/15eb7ki/no_more_waking_the_wife/
Error in expanding comments: name 'NoSuchElementException' is not defined
Processing 2/2: https://www.reddit.com/r/playstation/comments/18ra56f/bought_a_ps5_controller_for_my_gf_and_she_did/
Error in expanding comments: name 'NoSuchElementException' is not defined
                                                Article            Author  \
0     https://www.reddit.com/r/playstation/comments/...     KuntaWuKnicks   
1     https://www.reddit.com/r/playstation/comments/...   Ikewiththebeard   
2     https://www.reddit.com/r/playstation/comments/...     JamesJakes000   
3     https://www.reddit.com/r/playstation/comments/...          XscytheD   
4     https://www.reddit.com/r/playstation/comments/...              muws   
...                                                 ...               ...   
1308  https://www.reddit.com/r/playstation/comments/...    Tokenserious23   
1309  https://www.reddit

---
---
---

# Testing with itables 🧪

In [26]:
from itables import show

In [100]:
show(df_comments)